In [ ]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
data =load_dataset("thibautweber/fraud_detection5")
print(data)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 1000
    })
})


In [ ]:

print(data['train'][:5])


{'instruction': ["This transaction occurred on 2023-01-22 03:56:37. The amount of $8.3 was spent in the category 'grocery_pos' at the merchant 'fraud_Deckow-O'Conner' located in Port Patrick.", "At the time of this transaction, 2023-01-22 03:35:46. The amount of $266.84 was spent in the category 'gas_transport' at the merchant 'fraud_Kuvalis Ltd' located in Dominguezside.", "On this day, a transaction took place 2023-01-22 01:51:20. The amount of $291.05 was spent in the category 'grocery_pos' at the merchant 'fraud_Cartwright-Harris' located in Deannamouth.", "This transaction occurred on 2023-01-22 00:40:43. The amount of $307.71 was spent in the category 'grocery_pos' at the merchant 'fraud_Deckow-O'Conner' located in West Amyberg.", "Here are the details of the transaction that occurred 2023-01-22 13:06:24. The amount of $1105.5 was spent in the category 'shopping_net' at the merchant 'fraud_Gleason-Macejkovic' located in Nicolefort."], 'output': ['The transaction is Fraudulent bec

In [ ]:
import pandas as pd


df_train = pd.DataFrame(data['train'])


print(df_train.head())


                                         instruction  \
0  This transaction occurred on 2023-01-22 03:56:...   
1  At the time of this transaction, 2023-01-22 03...   
2  On this day, a transaction took place 2023-01-...   
3  This transaction occurred on 2023-01-22 00:40:...   
4  Here are the details of the transaction that o...   

                                              output  
0  The transaction is Fraudulent because it happe...  
1  The transaction is Fraudulent because it happe...  
2  The transaction is Fraudulent because it happe...  
3  The transaction is Fraudulent because it happe...  
4  Fraudulent because of the big expense, around ...  


In [ ]:

print(df_train.head())
print(df_train.columns.tolist())


                                         instruction  \
0  This transaction occurred on 2023-01-22 03:56:...   
1  At the time of this transaction, 2023-01-22 03...   
2  On this day, a transaction took place 2023-01-...   
3  This transaction occurred on 2023-01-22 00:40:...   
4  Here are the details of the transaction that o...   

                                              output  \
0  The transaction is Fraudulent because it happe...   
1  The transaction is Fraudulent because it happe...   
2  The transaction is Fraudulent because it happe...   
3  The transaction is Fraudulent because it happe...   
4  Fraudulent because of the big expense, around ...   

                                 cleaned_description  
0  transaction occurred 2023 01 22 03 56 37 amoun...  
1  time transaction 2023 01 22 03 35 46 amount 26...  
2  day transaction took place 2023 01 22 01 51 20...  
3  transaction occurred 2023 01 22 00 40 43 amoun...  
4  detail transaction occurred 2023 01 22 13 06 2..

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Function to clean text
def clean_text(text):
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    text = text.split()
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text if word not in stopwords.words('english')]
    return ' '.join(text)


df_train['cleaned_description'] = df_train['instruction'].apply(clean_text)


In [ ]:

print(df_train.columns.tolist())


['instruction', 'output', 'cleaned_description']


In [ ]:
data = load_dataset("thibautweber/fraud_detection5")
df_train = pd.DataFrame(data['train'])


def clean_text(text):
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    text = text.split()
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text if word not in stopwords.words('english')]
    return ' '.join(text)

In [ ]:
df_train['cleaned_description'] = df_train['instruction'].apply(clean_text)

In [ ]:
X = df_train['cleaned_description']
y = df_train['output']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train_tfidf, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
y_pred = model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred, zero_division=0))

                                                                                            precision    recall  f1-score   support

                                                                         Likely Fraudulent       0.00      0.00      0.00         1
                                                                                Fraudulent       0.00      0.00      0.00         4
                                Fraudulent beause the transaction occured during the night       0.00      0.00      0.00         3
                             Fraudulent because it is too much money for shopping at 11 PM       0.00      0.00      0.00         2
Fraudulent because of the big expense, $836 in shopping to a single shop during the night.       0.00      0.00      0.00         1
                               Likely Fraudulent beacause this is too much money at 10 PM        0.00      0.00      0.00         1
                                                                           